# BEE 4750 Lab 3: Linear Programming with JuMP

**Name**: Christine Swanson

**ID**: cms549

> **Due Date**
>
> Friday, 10/13/23, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [39]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `c:\Users\chris\Box\classwork\2023_Fall\BEE5750\labs\lab03-christinemswanson`


In [40]:
using JuMP # optimization modeling syntax
using HiGHS # optimization solver
using Plots # plotting

## Overview

In this lab, you will write and solve a resource allocation example
using `JuMP.jl`. `JuMP.jl` provides an intuitive syntax for writing,
solving, and querying optimization problems.

For an example of using `JuMP.jl` to solve linear programs, see [the
relevant tutorial on the class
website](https://viveks.me/environmental-systems-analysis/tutorials/julia-jump.html).

Free free to delete some of the illustrative cells and code blocks in
your notebook as you go through and solve the lab problems…this might
help reduce some potential confusion while grading about what your
answer is.

## Introduction

Your task is to decide how much lumber to produce to maximize profit
from wood sales. You can purchase wood from a managed forest, which
consists of spruce (320,000 bf) and fir (720,000 bf). Spruce costs
\$0.12 per bf to purchase and fir costs \$0.08 per bf.

At the lumber mill, wood can be turned into plywood of various grades
(see <a href="#tbl-inputs" class="quarto-xref">Table 1</a> for how much
wood of each type is required for and the revenue from each grade). Any
excess wood is sent to be recycled into particle board. This resource
allocation problem is diagrammed in
<a href="#fig-schematic" class="quarto-xref">Figure 1</a>.

| Plywood Grade | Inputs (bf/bf plywood) | Revenue (\$/1000 bf) |
|:-------------:|:----------------------:|:--------------------:|
|       1       |   0.5 (S) + 1.5 (F)    |         400          |
|       2       |   1.0 (S) + 2.0 (F)    |         520          |
|       3       |   1.5 (S) + 2.0 (F)    |         700          |

Table 1: Wood inputs and revenue by plywood grade. S refers to spruce
inputs, F fir inputs.

<figure id="fig-schematic">
<img src="attachment:lab03_files/figure-ipynb/mermaid-figure-1.png" />
<figcaption>Figure 1: Flowchat of the resource allocation problem in
this lab.</figcaption>
</figure>

## Problems (10 points)

### Problem 1: Problem Formulation (5 points)

In this problem, you will go through the steps of formulating a linear
program for this problem.

#### Problem 1.1 (1 point)

What are your decision variables? Clearly define your notation,
including what variables you are using, what they mean, and what their
units are.

**Decision Variables:** g1, g2, g3, g4, g5, g6. Here are the descriptions of the decision variables: 

g1: the amount of lumber produced from spruce for Grade 1-plywood, units in board feet 

g2: the amount of lumber produced from fir for Grade 1-plywood, units in board feet

g3: the amount of lumber produced from spruce for Grade 2-plywood, units in board feet

g4: the amount of lumber produced from fir for Grade 2-plywood, units in board feet

g5: the amount of lumber produced from spruce for Grade 3-plywood, units in board feet

g6: the amount of lumber produced from fir for Grade 3-plywood, units in board feet

#### Problem 1.2 (1 point)

Derive your objective function. Support your function with
justifications and/or equations as necessary. You will not receive
credit just for the function alone.

**Objective Function Derivation:** Our goal is to maximize profit from wood sales. So, the objective function can be written as: 

max(profit) = 0.4(0.5(g1) + 1.5(g2)) + 0.52(g3 + 2(g4)) + 0.70(1.5(g5) + 2(g6)) - 0.12(g1 + g3 + g5) - 0.08(g2 + g4 + g6)

This equation can be simplified to: 

0.08g1 + 0.53g2 + 0.4g3 + 0.96g4 + 0.93g5 + 1.32g6

So, I am essentally defining my function as Profit = Revenue - Cost. Thus, here, I am trying to take the sum of the revenue gained from each plywood grade, and subtract away the cost to purchase each board foot of spruce or fir. I converted all units to board feet, which is why we see 0.4, 0.52, and 0.7 for the revenue values. 

#### Problem 1.3 (2 point)

Derive any needed constraints. Support your function with justifications
and/or equations as necessary. You will not receive credit just for the
final constraints alone.

Based on the problem, we know there needs to be bounds on the supply of spruce and fir: 

g1 + g3 + g5 <= 320000 (spruce limit)

g2 + g4 + g6 <= 720000 (fir limit)

We also need a non-negativity constraint for our variables: 

g1, g2, g3, g4, g5, g6 > 0

#### Problem 1.4 (1 point)

Put this optimization problem in mathematical programming form. For an
example of the syntax for this, see lines 82–91
[here](https://github.com/vsrikrish/environmental-systems-analysis/blob/Fall23/tutorials/julia-jump.qmd).

$$\begin{equation}
\begin{aligned}
& \max_{g1, g2, g3, g4, g5, g6} & 0.4(0.5(g1) + 1.5(g2)) + 0.52(g3 + 2(g4)) + 0.7(1.5(g5) + 2(g6)) - 0.12(g1 + g3 + g5) - 0.08(g2 + g4 + g6)
\\
&\text{subject to} & \\
& & g1 + g3 + g5 \leq 320000\\
& & g2 + g4 + g6 \leq 720000\\
& & g1, g2, g3, g4, g5, g6 \geq 0\\ 
\end{aligned} 
\end{equation}$$

### Problem 2: Find the Solution (5 points)

#### Problem 2.1 (2 points)

Code your linear program using `JuMP`. Feel free to consult [the
website’s `JuMP`
tutorial](https://viveks.me/environmental-systems-analysis/tutorials/julia-jump.html)
for syntax help. The keys:

1.  Initialize your model with a solver; in this case, we’ll use the
    `HiGHS` solver, but there are other solvers listed here for
    different types of problems, some of which are open and some of
    which require a commercial license:
    <https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers>:

    ``` julia
    example_model = Model(HiGHS.Optimizer)
    ```

2.  Define variables with syntax like

    ``` julia
    @variable(example_model, 1 >= example_x >= 0)
    ```

    This will create a variable `example_x` which is constrained to be
    between 0 and 1; you can leave off any of the bounds if a variable
    is unbounded in a particular direction. You can also add a vector of
    variables:

    ``` julia
    T = 1:3 # define set to index variables
    @variable(example_model, 1 >= example_z[t in T] >= 0)
    ```

    which will create a vector of 3 variables `example_z[1]`, …,
    `example_z[3]`, all of which are bounded between 0 and 1.

3.  Add an objective with

    ``` julia
    @objective(example_model, Max, example_x + sum(example_z))
    ```

    which will add an objective to maximize (replace with `Min` to
    minimize).

4.  Add constraints:

    ``` julia
    @constraint(example_model, constraint1, 2example_x + 3*sum(example_z) <= 10)
    @constraint(example_model, constraint2, 5example_x - example_z[1] <= 2)
    ```

    which will name the constraints `constraint1` and `constraint2` (you
    should make yours more descriptive about what the constraint
    actually is). The value of adding a name is to facilitate later
    querying of shadow prices, which we will discuss later. You can also
    add a vector of constraints which have similar structure or rely on
    different elements of a data vector:

    ``` julia
    A = [2; 4]
    b = [8; 12]
    I = 1:2 # set indices for constraint
    @constraint(example_model, vector_constraint[i in I], A[i] * sum(example_z) .<= b[i])
    ```

    You can also define matrices of constraints which depend on two
    index sets by generalizing this syntax, e.g.

    ``` julia
    @constraint(example_model, matrix_constraint[i in I, j in J, ...])
    ```

    > **Tip**
    >
    > Specifying higher-dimensional vectors and matrices of variables
    > and constraints will be important when we start looking at more
    > complex applications, so don’t skip over this! You don’t want to
    > manually enter thousands of constraints to ensure hourly
    > electricity demand is met…

    Finally, you can (and probably should) `print` your model to make
    sure that you get something that looks like the equations that you
    wrote down (in a notebook, this will be nicely rendered):

    ``` julia
    print(example_model)
    ```

    $$ \begin{aligned}
    \max\quad & example\_x + example\_z_{1} + example\_z_{2} + example\_z_{3}\\
    \text{Subject to} \quad & 2 example\_x + 3 example\_z_{1} + 3 example\_z_{2} + 3 example\_z_{3} \leq 10\\
     & 5 example\_x - example\_z_{1} \leq 2\\
     & 2 example\_z_{1} + 2 example\_z_{2} + 2 example\_z_{3} \leq 8\\
     & 4 example\_z_{1} + 4 example\_z_{2} + 4 example\_z_{3} \leq 12\\
     & example\_x \geq 0\\
     & example\_z_{1} \geq 0\\
     & example\_z_{2} \geq 0\\
     & example\_z_{3} \geq 0\\
     & example\_x \leq 1\\
     & example\_z_{1} \leq 1\\
     & example\_z_{2} \leq 1\\
     & example\_z_{3} \leq 1\\
    \end{aligned} $$

    > **Define your entire model in one cell**
    >
    > `JuMP` has great and intuitive syntax, but it doesn’t like
    > re-defining variables or constraints once they’ve been set. I
    > recommend putting all of your model-definition code (starting from
    > `model = Model(...)`) for a particular optimization problem in a
    > single notebook cell, so you can re-set up the entire problem with
    > a single click when you want to make a change.

In [41]:
# code the linear program, implement the steps above

# step 1: initialize model with solver
lumber_profit_model = Model(HiGHS.Optimizer)

# step 2: define variables
T = 1:6
@variable(lumber_profit_model, g[t in T])

# step 3: define objective function
@objective(lumber_profit_model, Max, 
0.4*(0.5*(g[1]) + 1.5*(g[2])) + 0.52*(g[3] + 2*(g[4])) + 0.7*(1.5*(g[5]) + 2*(g[6])) - 
0.12*(g[1] + g[3] + g[5]) - 0.08*(g[2] + g[4] + g[6]))

# step 4: add constraints
@constraint(lumber_profit_model, spruce_constraint, g[1] + g[3] + g[5] <= 320000)
@constraint(lumber_profit_model, fir_constraint, g[2] + g[4] + g[6] <= 720000)
@constraint(lumber_profit_model, nonzero_constraint, g[1:6] >= 0) #non-zero constraint

# step 5: print the model
print(lumber_profit_model)

Max 0.08000000000000002 g[1] + 0.5200000000000001 g[2] + 0.4 g[3] + 0.9600000000000001 g[4] + 0.9299999999999998 g[5] + 1.3199999999999998 g[6]
Subject to
 spruce_constraint : g[1] + g[3] + g[5] <= 320000
 fir_constraint : g[2] + g[4] + g[6] <= 720000
 nonzero_constraint : [g[1], g[2], g[3], g[4], g[5], g[6]] in MathOptInterface.Nonnegatives(6)


#### Problem 2.2 (1 points)

Find the solution to your program and find the optimal values of the
decision variables. Once you’ve defined your model, you can find the
solution with \`optimize!():

In [42]:
optimize!(lumber_profit_model)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 6 cols, 6 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-8); columns 0(-6); elements 0(-12) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  1.2480000000e+06
HiGHS run time      :          0.00


> **What if I Get An Error?**
>
> If `optimize!()` throws an error, that’s usually a sign that something
> is wrong with the formulation (for example, a variable might not be
> bounded or a constraint might not be specified correctly) or a typo in
> the model definition. Linear programs should be well behaved!

To find the values of variables after optimizing, use `value.()` (the
broadcasting ensures this will work for vector-valued variables as
well):

In [43]:
print("g1 optimal value: ", value.(g[1]))

g1 optimal value: -0.0

In [44]:
print("g2 optimal value: ", value.(g[2]))

g2 optimal value: -0.0

In [45]:
print("g3 optimal value: ", value.(g[3]))

g3 optimal value: -0.0

In [46]:
print("g4 optimal value: ", value.(g[4]))

g4 optimal value: -0.0

In [47]:
print("g5 optimal value: ", value.(g[5]))

g5 optimal value: 320000.0

In [48]:
print("g6 optimal value: ", value.(g[6]))

g6 optimal value: 720000.0

In [49]:
# all vars to one vector
value.(g)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:6
And data, a 6-element Vector{Float64}:
     -0.0
     -0.0
     -0.0
     -0.0
 320000.0
 720000.0

#### Problem 2.3 (1 point)

How would your profit change if you could buy 1,000 additional bf of
spruce? You can answer this by getting the shadow price of a particular
variable with:

In [50]:
shadow_price(spruce_constraint) 

0.9299999999999998

In [51]:
shadow_price(fir_constraint) 

1.3199999999999998

**Response:** Thus, because my code above is in bf, my profit would change by approximately $930 if I could buy 1,000 additional bf of spruce. This is because 0.93*1000 = 930.   

#### Problem 2.4 (1 point)

Would you prefer to have 2,000 additional bf of spruce or 1,000
additional bf of fir?

In [52]:
shadow_price(spruce_constraint)*2 # 2K more bf spruce

1.8599999999999997

In [53]:
shadow_price(fir_constraint) # 1K more bf fir

1.3199999999999998

**Response**: I would prefer 2,000 additional bf of spruce as opposed to 1,000 additional bf of fir because 1.86 > 1.32. So, my profit would change by approximately 1,860 dollars if I could buy 2,000 more bf of spruce, as opposed to only changing by 1,320 dollars for 1,000 additional bf of fir. 

## References

Put any consulted sources here, including classmates you worked with/who
helped you.

I consulted with Yifan Luo to set up problem 1 of this lab. I also asked Gaby questions as I worked on the assignment. 

I also looked at the tutorial on the course website on Linear Programs before starting this lab. 